In [ ]:
from nltk.stem import PorterStemmer

def find_suffix_word(word, suffix):

  stemmer = PorterStemmer()
  stemmed_word = stemmer.stem(word)

  if stemmed_word.endswith(suffix):
      return stemmed_word
  else:
      return None

word = "running"
suffix = "run"

result = find_suffix_word(word, suffix)

if result:
  print(f"The stemmed word with suffix '{suffix}' is: {result}")
else:
  print(f"The suffix '{suffix}' was not found in the stemmed word of '{word}'.")


word = "happiness"
suffix = "happy"
result = find_suffix_word(word, suffix)

if result:
  print(f"The stemmed word with suffix '{suffix}' is: {result}")
else:
  print(f"The suffix '{suffix}' was not found in the stemmed word of '{word}'.")

The stemmed word with suffix 'run' is: run
The suffix 'happy' was not found in the stemmed word of 'happiness'.


In [ ]:
# prompt: generate a simple code , with importing the tokenizer , BPE , BpeTrainer, Whitespace and prepare a small dataset with indian lan and initialize the tooooekenizer and train the tokenizer =on the text and toeknize a new text

!pip install tokenizers

from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace

# Sample Indian language text dataset (replace with your actual data)
dataset = [
    "नमस्ते दुनिया",  # Hindi
    "வணக்கம் உலகம்", # Tamil
    "नमस्कार संसार", # Marathi
    "ഹലോ ലോകം", # Malayalam
]


# Initialize the tokenizer
tokenizer = Tokenizer(BPE(unk_token="[UNK]"))
tokenizer.pre_tokenizer = Whitespace()


# Initialize the trainer
trainer = BpeTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])


# Train the tokenizer
tokenizer.train_from_iterator(dataset, trainer=trainer)


# Tokenize a new text
new_text = "नमस्ते भारत"
encoded = tokenizer.encode(new_text)

encoded.tokens

['नमस्ते', '[UNK]', 'ार', 'त']

In [ ]:
# prompt: generate a code that imports from transforms are autotransformer and automodelfortokenclassifcation and pipeline  and load the indicbert tokenizer and model and load text in hindi and tokenize using the indicbert

!pip install transformers
!pip install indic-nlp-library

from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline

# Load the IndicBERT tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indic-bert")
model = AutoModelForTokenClassification.from_pretrained("ai4bharat/indic-bert")


# Hindi text
hindi_text = "मैं आज बाज़ार जा रहा हूँ।"

# Tokenize the Hindi text
tokens = tokenizer(hindi_text, return_tensors="pt")

# You can now use the 'tokens' with the model for further processing
# For example, for token classification:

# Perform token classification
# outputs = model(**tokens)  # Uncomment if you intend to use the model for classification


# Print the tokens
tokens

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 60.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 9.0 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/507 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/5.65M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/135M [00:00<?, ?B/s]

Some weights of AlbertForTokenClassification were not initialized from the model checkpoint at ai4bharat/indic-bert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'input_ids': tensor([[    2,  2092,   298, 18142,   367,  4765,  2022,  4384,    15,     3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [ ]:
from indicnlp.tokenize import sentence_tokenize
from transformers import pipeline, AutoTokenizer
import re

# Load the language detection pipeline
lang_detector = pipeline("text-classification", model="papluca/xlm-roberta-base-language-detection")

# Load the IndicBERT tokenizer (or any other suitable tokenizer)
tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indic-bert")

def process_text(text):
  """
  Processes the input text by detecting language, segmenting sentences,
  and performing word tokenization.

  Args:
    text: The input text string.

  Returns:
    A list of dictionaries. Each dictionary contains the detected language,
    segmented sentences, and tokens for a sentence. Returns an empty list if
    language detection fails or text is empty.
  """

  if not text:  # Handle empty text input
    return []

  try:
    detected_language = lang_detector(text)[0]["label"]
  except Exception as e:  # Handle potential errors in language detection
    print(f"Error during language detection: {e}")
    return []

  sentences = sentence_tokenize.sentence_split(text, lang=detected_language)

  results = []
  for sentence in sentences:
    # Handle cases where sentence might be None or empty
    if sentence:
      tokens = tokenizer.tokenize(sentence)
      results.append({
          "language": detected_language,
          "sentence": sentence,
          "tokens": tokens
      })
  return results

# Example usage:
text = "नमस्ते दुनिया। यह एक परीक्षण वाक्य है! कैसे हैं आप?  வணக்கம் உலகம்."

processed_output = process_text(text)
for item in processed_output:
    print(item)

text2 = "" # Testing empty string
processed_output = process_text(text2)
print(processed_output)

text3 = "This is an English sentence. Another one!" #Testing a different language
processed_output = process_text(text3)
for item in processed_output:
    print(item) # Indented this line to be inside the for loop

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Device set to use cpu


{'language': 'hi', 'sentence': 'नमस्ते दुनिया।', 'tokens': ['▁नम', 'सत', '▁', 'दन', 'य', '।']}
{'language': 'hi', 'sentence': 'यह एक परीक्षण वाक्य है!', 'tokens': ['▁यह', '▁एक', '▁पर', 'क', 'षण', '▁व', 'क', 'य', '▁ह', '!']}
{'language': 'hi', 'sentence': 'कैसे हैं आप?', 'tokens': ['▁कस', '▁ह', '▁आप', '?']}
{'language': 'hi', 'sentence': 'வணக்கம் உலகம்.', 'tokens': ['▁வ', 'ண', 'க', 'கம', '▁உலகம', '.']}
[]
{'language': 'en', 'sentence': 'This is an English sentence.', 'tokens': ['▁this', '▁is', '▁an', '▁eng', 'lish', '▁sentence', '.']}
{'language': 'en', 'sentence': 'Another one!', 'tokens': ['▁another', '▁one', '!']}


In [ ]:
!pip install kroki
!pip install kroki-mermaid # Install the required kroki-mermaid package

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.8/202.8 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 31.7 MB/s eta 0:00:00
  Created wheel for kroki: filename=kroki-0.1.2-py3-none-any.whl size=9065 sha256=834f1d6774a266605f836a05b516ce1b0686bb2bb95e2131d144235bfef376bd
  Stored in directory: /root/.cache/pip/wheels/5e/31/e5/3be72bc7a0710c313efdfdf6b5616598741bd49ef9735b435b
Successfully built kroki
ERROR: Could not find a version that satisfies the requirement kroki-mermaid (from versions: none)
ERROR: No matching distribution found for kroki-mermaid


In [ ]:
def quick_sort(arr):
    if len(arr) <= 1:
        return arr

    pivot = arr[len(arr) // 2]
    left = [x for x in arr if x < pivot]
    middle = [x for x in arr if x == pivot]
    right = [x for x in arr if x > pivot]

    return quick_sort(left) + middle + quick_sort(right)

# Example usage:
arr = [3, 6, 8, 10, 1, 2, 1]
sorted_arr = quick_sort(arr)
print(sorted_arr)


[1, 1, 2, 3, 6, 8, 10]


In [2]:
!pip install indic_nlp_library #Installing the correct indicnlp package

from nltk.stem import PorterStemmer
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
from indicnlp.tokenize import sentence_tokenize
from transformers import pipeline, AutoTokenizer
import re
import pandas as pd

# Stemming and Suffix Finding Function
def find_suffix_word(word, suffix):
  stemmer = PorterStemmer()
  stemmed_word = stemmer.stem(word)
  if stemmed_word.endswith(suffix):
    return stemmed_word
  else:
    return None

# Example Usage
word = "running"
suffix = "run"
result = find_suffix_word(word, suffix)
if result:
  print(f"The stemmed word with suffix '{suffix}' is: {result}")
else:
  print(f"The suffix '{suffix}' was not found in the stemmed word of '{word}'.")

word = "happiness"
suffix = "happy"
result = find_suffix_word(word, suffix)
if result:
  print(f"The stemmed word with suffix '{suffix}' is: {result}")
else:
  print(f"The suffix '{suffix}' was not found in the stemmed word of '{word}'.")

# Install required libraries
!pip install tokenizers
!pip install transformers
!pip install indic-nlp-library

# Sample Indian language text dataset
dataset = [
    "नमस्ते दुनिया",  # Hindi
    "வணக்கம் உலகம்", # Tamil
    "नमस्कार संसार", # Marathi
    "ഹലോ ലോകം", # Malayalam
]

# Initialize and train the tokenizer
tokenizer = Tokenizer(BPE(unk_token="[UNK]"))
tokenizer.pre_tokenizer = Whitespace()
trainer = BpeTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])
tokenizer.train_from_iterator(dataset, trainer=trainer)

# Tokenize a new text
new_text = "नमस्ते भारत"
encoded = tokenizer.encode(new_text)
print(encoded.tokens)

# Load the IndicBERT tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indic-bert")
model = AutoModelForTokenClassification.from_pretrained("ai4bharat/indic-bert")

# Hindi text example
hindi_text = "मैं आज बाज़ार जा रहा हूँ।"

# Tokenize the Hindi text
tokens = tokenizer(hindi_text, return_tensors="pt")

# Print the tokens
print(tokens)

# Load the language detection pipeline
lang_detector = pipeline("text-classification", model="papluca/xlm-roberta-base-language-detection")

# Function to process text (language detection, sentence segmentation, tokenization)
def process_text(text):
  if not text:
    return []

  try:
    detected_language = lang_detector(text)[0]["label"]
  except Exception as e:
    print(f"Error during language detection: {e}")
    return []

  sentences = sentence_tokenize.sentence_split(text, lang=detected_language)
  results = []
  for sentence in sentences:
    if sentence:
      tokens = tokenizer.tokenize(sentence)
      results.append({
          "language": detected_language,
          "sentence": sentence,
          "tokens": tokens
      })
  return results

# Example Usage
text = "नमस्ते दुनिया। यह एक परीक्षण वाक्य है! कैसे हैं आप?  வணக்கம் உலகம்."
processed_output = process_text(text)
for item in processed_output:
  print(item)

text2 = "" # Testing empty string
processed_output = process_text(text2)
print(processed_output)

text3 = "This is an English sentence. Another one!" #Testing a different language
processed_output = process_text(text3)
for item in processed_output:
    print(item)

# Install kroki
!pip install kroki
!pip install kroki-mermaid

# Quick Sort Function
def quick_sort(arr):
  if len(arr) <= 1:
      return arr

  pivot = arr[len(arr) // 2]
  left = [x for x in arr if x < pivot]
  middle = [x for x in arr if x == pivot]
  right = [x for x in arr if x > pivot]

  return quick_sort(left) + middle + quick_sort(right)

# Example usage
arr = [3, 6, 8, 10, 1, 2, 1]
sorted_arr = quick_sort(arr)
print(sorted_arr)


# Sentiment Analysis Function (Illustrative)
def analyze_sentiment(text):
  """
  A placeholder function to demonstrate how sentiment analysis would fit.
  In a real implementation, we'd use a pre-trained model.
  """
  # Use a simple placeholder model, replace it with an actual one from transformers
  sentiment_pipeline = pipeline("sentiment-analysis")
  sentiment = sentiment_pipeline(text)[0]
  return sentiment


# Example with sentiment analysis
text_for_sentiment = "This is a great day!"
sentiment_result = analyze_sentiment(text_for_sentiment)
print(f"Sentiment analysis result: {sentiment_result}")

text_for_sentiment = "This is horrible."
sentiment_result = analyze_sentiment(text_for_sentiment)
print(f"Sentiment analysis result: {sentiment_result}")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 10.4 MB/s eta 0:00:00
The stemmed word with suffix 'run' is: run
The suffix 'happy' was not found in the stemmed word of 'happiness'.
['नमस्ते', '[UNK]', 'ार', 'त']


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/507 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/5.65M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/135M [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cuda:0
No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cuda:0


Sentiment analysis result: {'label': 'POSITIVE', 'score': 0.9998801946640015}
Sentiment analysis result: {'label': 'NEGATIVE', 'score': 0.9997575879096985}


Dealing_With_Emojis

In [3]:
import re

def remove_emojis(text):
  emoji_pattern = re.compile(
      "["
      "\U0001F600-\U0001F64F"
      "\U0001F300-\U0001F5FF"
      "\U0001F680-\U0001F6FF"
      "\U0001F1E0-\U0001F1FF"
      "\U00002702-\U000027B0"
      "\U000024C2-\U0001F251"
      "]+",
      flags=re.UNICODE,
  )
  return emoji_pattern.sub(r"", text)

text_with_emojis = "Hello 😊 world! 🎉🌍"
text_without_emojis = remove_emojis(text_with_emojis)
print(f"Original text: {text_with_emojis}")
print(f"Text without emojis: {text_without_emojis}")


Original text: Hello 😊 world! 🎉🌍
Text without emojis: Hello  world! 


In [5]:
!pip install emot

import re
from emot.emo_unicode import UNICODE_EMOJI
from emot.emo_unicode import EMOTICONS_EMO

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 2.9 MB/s eta 0:00:00


In [11]:
def convert_emojis(text):
    for emot in UNICODE_EMOJI:
            text = text.replace(emot, "_".join(UNICODE_EMOJI[emot].replace(",","").replace(":","").split()))
    return text


convert_emojis(text)

'Hello smiling_face_with_smiling_eyes world! party_popperglobe_showing_Europe-Africa'

In [9]:
import re
from emot.emo_unicode import UNICODE_EMOJI

def convert_emojis(text):
    return ' '.join([UNICODE_EMOJI.get(char, char) for char in text])

# Example
text = "Hello 😊 world! 🎉🌍"
converted_text = convert_emojis(text)
converted_text


'H e l l o   :smiling_face_with_smiling_eyes:   w o r l d !   :party_popper: :globe_showing_Europe-Africa:'

Data Frames

In [12]:


import pandas as pd
from emot.emo_unicode import UNICODE_EMOJI
import re

data = {'text': ["Hello 😊 world! 🎉🌍", "This is a sad 😞 message", "I am happy 😄", "No emojis here"]}
df = pd.DataFrame(data)

print(df)


def convert_emojis(text):
  return ' '.join([UNICODE_EMOJI.get(char, char) for char in text])


df['text_emojis_converted'] = df['text'].apply(convert_emojis)

df


                      text
0        Hello 😊 world! 🎉🌍
1  This is a sad 😞 message
2             I am happy 😄
3           No emojis here


,text,text_emojis_converted
0,Hello 😊 world! 🎉🌍,H e l l o :smiling_face_with_smiling_eyes: ...
1,This is a sad 😞 message,T h i s i s a s a d :disappointed_face...
2,I am happy 😄,I a m h a p p y :grinning_face_with_smil...
3,No emojis here,N o e m o j i s h e r e
